<a href="https://colab.research.google.com/github/tho071206-ther/TriTueNhanTao/blob/main/minimax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bai thuc hanh tuan 3

In [4]:
import math
NGUOI_CHOI_MAX = 'X' # Người chơi MAX (AI)
DOI_THU_MIN = 'O'    # Người chơi MIN (Đối thủ)
DIEM_THANG = 1000000 # Điểm rất cao cho trạng thái thắng


def get_board_size():
    # Hàm cho phép người dùng nhập kích thước N của bàn cờ N*N (N >= 3).
    while True:
        try:
            n = int(input("NHẬP: Kích thước bàn cờ N*N (ví dụ 3, 5, 10): "))
            if n >= 3:
                return n
            else:
                print("LỖI! Vui lòng nhập N >= 3!")
        except ValueError:
            print("LỖI! Vui lòng nhập một số nguyên!")

def check_win(ban_co, nguoi_choi):
    """ Kiểm tra xem 'nguoi_choi' có đạt N quân liên tiếp trên bàn cờ N*N hay không."""
    N = len(ban_co)

    # 1. Kiểm tra Ngang
    for r in range(N):
        if all(ban_co[r][c] == nguoi_choi for c in range(N)):
            return True

    # 2. Kiểm tra Dọc
    for c in range(N):
        if all(ban_co[r][c] == nguoi_choi for r in range(N)):
            return True

    # 3. Kiểm tra Chéo Chính (\)
    if all(ban_co[i][i] == nguoi_choi for i in range(N)):
        return True

    # 4. Kiểm tra Chéo Phụ (/)
    if all(ban_co[i][N - 1 - i] == nguoi_choi for i in range(N)):
        return True

    return False

def is_terminal(ban_co):
    """Kiểm tra xem bàn cờ đã kết thúc (thắng, thua, hòa) hay vẫn tiếp tục."""
    # 1. Kiểm tra thắng/thua
    if check_win(ban_co, NGUOI_CHOI_MAX) or check_win(ban_co, DOI_THU_MIN):
        return True

    # 2. Kiểm tra hòa (hết ô trống)
    for hang in ban_co:
        if ' ' in hang:
            return False # Vẫn còn ô trống
    return True # Bàn cờ đầy, trạng thái hòa

def evaluate(ban_co):
    """Hàm đánh giá Heuristic đơn giản (chỉ tính thắng, thua, hòa)."""
    if check_win(ban_co, NGUOI_CHOI_MAX):
        return DIEM_THANG     # AI thắng: Điểm rất cao
    if check_win(ban_co, DOI_THU_MIN):
        return -DIEM_THANG   # Đối thủ thắng: Điểm rất thấp
    return 0                 # Hòa hoặc thế cờ chưa rõ ràng

def get_available_moves(ban_co):
    """Tìm tất cả các nước đi hợp lệ (ô trống)."""
    moves = []
    N = len(ban_co)
    for hang in range(N):
        for cot in range(N):
            if ban_co[hang][cot] == ' ':
                moves.append((hang, cot))
    return moves

def in_ban_co(ban_co):
    """Hàm in trạng thái bàn cờ."""
    N = len(ban_co)
    print("\n   " + " ".join(str(i) for i in range(N)))
    print("  " + "---" * N)
    for i in range(N):
        print(f"{i} | " + " ".join(ban_co[i]))
    print("  " + "---" * N)
def minimax_thuan(ban_co, do_sau, la_luot_toi_da_hoa):
    """
    Tham số:
    - ban_co: Trạng thái bàn cờ hiện tại.
    - do_sau: Độ sâu còn lại để tìm kiếm.
    - la_luot_toi_da_hoa: True nếu là lượt của AI (MAX), False nếu là lượt của Đối thủ (MIN).
    """

    # ĐIỀU KIỆN DỪNG: Đạt độ sâu giới hạn HOẶC Trạng thái kết thúc
    if do_sau == 0 or is_terminal(ban_co):
        return evaluate(ban_co)

    if la_luot_toi_da_hoa:
        # LƯỢT CỦA MAX (AI) - Tìm giá trị lớn nhất
        gia_tri_toi_da = -math.inf

        for hang, cot in get_available_moves(ban_co):
            # 1. Thực hiện nước đi ảo
            ban_co[hang][cot] = NGUOI_CHOI_MAX

            # 2. Gọi đệ quy cho lượt của MIN
            gia_tri = minimax_thuan(ban_co, do_sau - 1, False)

            # 3. Hoàn trả trạng thái (Backtrack)
            ban_co[hang][cot] = ' '

            # 4. Cập nhật giá trị tốt nhất (tối đa hóa)
            gia_tri_toi_da = max(gia_tri_toi_da, gia_tri)

        return gia_tri_toi_da

    else:
        # LƯỢT CỦA MIN (Đối thủ) - Tìm giá trị nhỏ nhất
        gia_tri_toi_thieu = math.inf

        for hang, cot in get_available_moves(ban_co):
            # 1. Thực hiện nước đi ảo
            ban_co[hang][cot] = DOI_THU_MIN

            # 2. Gọi đệ quy cho lượt của MAX
            gia_tri = minimax_thuan(ban_co, do_sau - 1, True)

            # 3. Hoàn trả trạng thái (Backtrack)
            ban_co[hang][cot] = ' '

            # 4. Cập nhật giá trị tốt nhất (tối thiểu hóa)
            gia_tri_toi_thieu = min(gia_tri_toi_thieu, gia_tri)

        return gia_tri_toi_thieu

def find_best_move_minimax(ban_co, do_sau):
    """Hàm gọi Minimax để tìm nước đi tối ưu nhất cho AI (MAX)."""
    best_eval = -math.inf
    best_move = None

    for move in get_available_moves(ban_co):
        hang, cot = move

        ban_co[hang][cot] = NGUOI_CHOI_MAX # Thử nước đi

        # Gọi Minimax thuần. Lượt kế tiếp là lượt của Đối thủ (MIN).
        eval = minimax_thuan(ban_co, do_sau - 1, False)

        ban_co[hang][cot] = ' ' # Hoàn trả trạng thái

        # Nếu tìm thấy giá trị tốt hơn, cập nhật
        if eval > best_eval:
            best_eval = eval
            best_move = move

    return best_move, best_eval

if __name__ == '__main__':
    N = get_board_size()
    ban_co = [[' ' for _ in range(N)] for _ in range(N)]
    search_depth = min(4, N*N) # Giới hạn độ sâu

    print("\n--- PHIÊN BẢN 1: THUẬT TOÁN MINIMAX THUẦN ---")
    print(f"Kích thước: {N}x{N}, Độ sâu tìm kiếm: {search_depth}")
    in_ban_co(ban_co)

    # Giả lập một bước đi đầu tiên của AI
    print(f"\n[AI đang suy nghĩ với Minimax thuần (Độ sâu = {search_depth})]...")

    best_move, score = find_best_move_minimax(ban_co, search_depth)

    if best_move:
        hang, cot = best_move
        ban_co[hang][cot] = NGUOI_CHOI_MAX
        print(f"-> Nước đi tối ưu (Minimax thuần): ({hang}, {cot})")
        print(f"-> Giá trị thế cờ (Score): {score}")
        in_ban_co(ban_co)
    else:
        print("Không còn nước đi nào.")

NHẬP: Kích thước bàn cờ N*N (ví dụ 3, 5, 10): 3

--- PHIÊN BẢN 1: THUẬT TOÁN MINIMAX THUẦN ---
Kích thước: 3x3, Độ sâu tìm kiếm: 4

   0 1 2
  ---------
0 |      
1 |      
2 |      
  ---------

[AI đang suy nghĩ với Minimax thuần (Độ sâu = 4)]...
-> Nước đi tối ưu (Minimax thuần): (0, 0)
-> Giá trị thế cờ (Score): 0

   0 1 2
  ---------
0 | X    
1 |      
2 |      
  ---------
